In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sqlalchemy import create_engine


import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
housing_df = pd.read_sql_query('select * from houseprices',con=engine)


engine.dispose()

In [2]:
#Setting dummy columns

housing_df = pd.concat([housing_df,pd.get_dummies(housing_df.kitchenqual, prefix='kitchenqual', drop_first=True)], axis=1)
housing_df = pd.concat([housing_df,pd.get_dummies(housing_df.exterqual, prefix='exterqual', drop_first=True)], axis=1)
housing_df = pd.concat([housing_df,pd.get_dummies(housing_df.mszoning, prefix='mszoning', drop_first=True)], axis=1)

dummy_column_names = list(pd.get_dummies(housing_df.kitchenqual, prefix='kitchenqual', drop_first=True).columns)
dummy_column_names = dummy_column_names + list(pd.get_dummies(housing_df.mszoning, prefix='mszoning', drop_first=True).columns)
dummy_column_names = dummy_column_names + list(pd.get_dummies(housing_df.exterqual, prefix='exterqual', drop_first=True).columns)

In [3]:
#Setting additional variables for model: interaction term between overall qual and first floor square feet
housing_df['int_overallqual_firstflrsf'] = housing_df.overallqual * housing_df.firstflrsf
X = housing_df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'int_overallqual_firstflrsf', 'firstflrsf'] + dummy_column_names]
Y = housing_df.saleprice

In [4]:
#Splitting data into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 17)

print("The number of observations in training set is {}".format(X_train.shape[0]))
print("The number of observations in test set is {}".format(X_test.shape[0]))

alphas = [np.power(10.0,p) for p in np.arange(-10,40,1)]

The number of observations in training set is 1168
The number of observations in test set is 292


In [5]:
#OLS

lrm = LinearRegression()
lrm.fit(X_train, y_train)


# We are making predictions here
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in the training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in the test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in the training set is: 0.7875970696698855
-----Test set statistics-----
R-squared of the model in the test set is: 0.8401978887324437
Mean absolute error of the prediction is: 22740.087031692732
Mean squared error of the prediction is: 1064352182.5504053
Root mean squared error of the prediction is: 32624.410838364656
Mean absolute percentage error of the prediction is: 13.513311761400068


In [6]:
#Ridge

ridgeregr = RidgeCV(alphas=alphas, cv=10) 
ridgeregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridgeregr.predict(X_train)
y_preds_test = ridgeregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(ridgeregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(ridgeregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


R-squared of the model on the training set is: 0.7874483330568419
-----Test set statistics-----
R-squared of the model on the test set is: 0.8392727184824031
Mean absolute error of the prediction is: 22837.06184581206
Mean squared error of the prediction is: 1070514222.3823627
Root mean squared error of the prediction is: 32718.713641926126
Mean absolute percentage error of the prediction is: 13.589581149607374


In [7]:
#Lasso

lassoregr = LassoCV(alphas=alphas, cv=10)
lassoregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lassoregr.predict(X_train)
y_preds_test = lassoregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(lassoregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(lassoregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


R-squared of the model on the training set is: 0.7875521274451533
-----Test set statistics-----
R-squared of the model on the test set is: 0.8397730634750555
Mean absolute error of the prediction is: 22783.674713886354
Mean squared error of the prediction is: 1067181705.1788435
Root mean squared error of the prediction is: 32667.747170241837
Mean absolute percentage error of the prediction is: 13.55769222173574


In [8]:
#ElasticNet

elasticregr = ElasticNetCV(alphas=alphas, cv=10)
elasticregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticregr.predict(X_train)
y_preds_test = elasticregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(elasticregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(elasticregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


R-squared of the model on the training set is: 0.7875364728476019
-----Test set statistics-----
R-squared of the model on the test set is: 0.8396370346533881
Mean absolute error of the prediction is: 22799.03832223858
Mean squared error of the prediction is: 1068087716.8208865
Root mean squared error of the prediction is: 32681.611294746264
Mean absolute percentage error of the prediction is: 13.561152804723726


The best performing model is OLS.  This model has the largest R-squared of the training and test sets, but also has the lowest evaluation metrics.   